In [ ]:
import os

from pathlib import Path
from datasets import load_dataset, Dataset

from data_processing.preprocessors.preprocessor import PreprocessingPipeline
from data_processing.preprocessors.preprocessor import RemoveComments
from data_processing.preprocessors.preprocessor import SyntaxValidator
from data_processing.preprocessors.preprocessor import DuplicateFilter
from data_processing.preprocessors.preprocessor import Pep8Formatter

from data_processing.pretokenizers.firstpretokenizer import FirstPretokenizer

from data_processing.utils.pretokenize_all import pretokenize_all
from data_processing.utils.docstring_and_code_filtering import doctring_and_code_filtering
from data_processing.utils.data_loader import load_and_split_dataset
from data_processing.utils.data_preparation import preprocess

from model_operations.utils.model_utils import load_tokenizer

from config import (
    RUN_SEGEMENTATOR,
    MODEL_NAME,
    MAX_INPUT_LENGTH,
    MAX_OUTPUT_LENGTH,
    USE_CUSTOM_EOS,
    EOS,
)

/home/patryk/Documents/syntax-aware-language-model-for-code-generation/model_operations/training/models/t5-base-split1-epochs1-lossFalse


In [3]:
access_token = os.getenv("HUGGINGFACE_TOKEN")

# Intro

This is a simple yet powerfull step by step guideline for reproducing our results. The repository contains code for experimenting with a custom training method for the T5 language model, aimed at improving its performance in code generation tasks.

# Dataset preperation

Download and truncate the raw dataset to the desired size.

In [19]:
dataset = load_dataset("Nan-Do/code-search-net-python", token=access_token)["train"]
dataset = dataset.select(range(10000))
print("Number of samples before cleaning:", len(dataset))

Number of samples before cleaning: 10000


Clean the samples by removing comments, formatting them to the PEP8 standard, and getting rid of syntactically incorrect samples and duplicates.

In [20]:
pipeline = PreprocessingPipeline([
    RemoveComments(),
    DuplicateFilter(),
    Pep8Formatter(),
    SyntaxValidator()
])

data = pipeline.apply(dataset)

print("Number of samples after cleaning:", len(data))

Number of samples after cleaning: 9899


Load and apply the pretokenizer.

The pretokenizer is simply an AST visitor that walks through the code structure and changes its string representation to match special tokens in the tokenizer, so they can be detected and translated into the proper token IDs

In [21]:
pretokenizer = FirstPretokenizer(_use_dedent=True, _use_semantics=True)
data = pretokenize_all(data, pretokenizer)

<unknown>:14: SyntaxWarning: invalid escape sequence '\s'
<unknown>:11: SyntaxWarning: invalid escape sequence '\Z'
<unknown>:3: SyntaxWarning: invalid escape sequence '\('
<unknown>:3: SyntaxWarning: invalid escape sequence '\('
<unknown>:3: SyntaxWarning: invalid escape sequence '\.'
<unknown>:4: SyntaxWarning: invalid escape sequence '\}'
<unknown>:4: SyntaxWarning: invalid escape sequence '\.'
<unknown>:4: SyntaxWarning: invalid escape sequence '\.'
<unknown>:3: SyntaxWarning: invalid escape sequence '\('
<unknown>:13: SyntaxWarning: invalid escape sequence '\s'


Load and apply the semgmentator if specified.

It allows for "masking" and helps the model capture local dependencies. It replaces each sample with a few new ones that have consistent code fragments masked out, and the labels are adjusted accordingly

In [22]:
if RUN_SEGEMENTATOR:
    data = ...

Add custom eos token if specified.

In [23]:
if USE_CUSTOM_EOS:
    for sample in data:
        sample['parsed'] += EOS

Let's filter out unnecessary columns.

In [ ]:
data = doctring_and_code_filtering(data)

Samples with ≤512 tokens: 9081 / 9899 (91.74%)


In [ ]:
# temporary 
with open("docstring_and_code.jsonl", 'w') as f:
    for sample in data:
        f.write(f"{sample}\n")

Split the data into subsets.

In [ ]:
dataset_dict = load_and_split_dataset(data)

{'train': Dataset({
     features: ['docstring', 'parsed'],
     num_rows: 748
 }),
 'validation': Dataset({
     features: ['docstring', 'parsed'],
     num_rows: 94
 }),
 'test': Dataset({
     features: ['docstring', 'parsed'],
     num_rows: 94
 })}

The last step to complete data preparation is to adjust the sample length to the model's context window. We need to **load a tokenizer**.

In [11]:
tokenizer, specifics = load_tokenizer(MODEL_NAME, USE_CUSTOM_EOS, pretokenizer)
if specifics:
    semantic_start_id, semantic_end_id, code_token_ids, semantic_token_ids = specifics

Perform tokenization and length adjustments.

In [12]:
tokenized_dataset = {
    split: dataset.map(
        lambda batch: preprocess(batch, tokenizer, USE_CUSTOM_EOS, MAX_INPUT_LENGTH, MAX_OUTPUT_LENGTH),
        batched=True,
        remove_columns=dataset.column_names
    )
    for split, dataset in dataset_dict.items()
}

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

Map:   0%|          | 0/94 [00:00<?, ? examples/s]

Map:   0%|          | 0/94 [00:00<?, ? examples/s]